# Kernel Calculation Test
This notebook is used to test the functionality of the kernel calc methods that (if working) will be rewritten in Python script.

In [2]:
import torch
import resources as rs

### Loading Model Data

In [3]:
model_path = 'ex_models.pt'
state_dict = torch.load(model_path, map_location=torch.device('cpu')) # MODEL

### Unpacking the params

**NOTE:** 
The model was trained on the MNIST digits of 0 1

Unpacking the features params

In [4]:
import torch.nn as nn

# get the weights and biases of the quantized model (for the features layer)
f_weights_quant = state_dict['features.hidden_layer._packed_params._packed_params'][0]
f_bias_quant = state_dict['features.hidden_layer._packed_params._packed_params'][1]

# dequantize the weights and biases
f_weights_float = torch.dequantize(f_weights_quant)
f_bias_float = torch.dequantize(f_bias_quant)
# print the float values of weights and biases
print(f_weights_float)
print(f_bias_float)

tensor([[ 0.0212, -0.0311, -0.0050,  ..., -0.0343, -0.0193,  0.0000],
        [ 0.0112, -0.0050,  0.0149,  ..., -0.0093, -0.0106,  0.0336],
        [-0.0324, -0.0006, -0.0062,  ...,  0.0311,  0.0137, -0.0324],
        ...,
        [-0.0305,  0.0168, -0.0293,  ...,  0.0237, -0.0280, -0.0112],
        [-0.0349,  0.0056, -0.0174,  ..., -0.0093, -0.0031,  0.0212],
        [ 0.0349,  0.0168, -0.0206,  ..., -0.0149,  0.0044, -0.0187]])
Parameter containing:
tensor([ 0.0235, -0.0103, -0.0210,  ..., -0.0244,  0.0028, -0.0313],
       grad_fn=<NotImplemented>)


Unpacking the readout params.

In [5]:
# get the weights and biases of the quantized model (for the readout layer)
r_weights_quant = state_dict['readout._packed_params._packed_params'][0]
r_bias_quant = state_dict['readout._packed_params._packed_params'][1]

# dequantize the weights and bises
r_weights_float = torch.dequantize(r_weights_quant)
r_bias_float = torch.dequantize(r_bias_quant)

print(r_weights_float)
print(r_bias_float)

tensor([[ 0.0219,  0.0389,  0.0024,  ..., -0.0073,  0.0182, -0.0158],
        [-0.0195, -0.0219, -0.0024,  ...,  0.0073, -0.0170,  0.0134]])
Parameter containing:
tensor([0.4623, 0.4824], grad_fn=<NotImplemented>)


### Manually updating the model

In [6]:
model = rs.NN()
model

NN(
  (features): Sequential(
    (hidden_layer): Linear(in_features=784, out_features=2048, bias=True)
    (hidden_activation): ReLU()
  )
  (readout): Linear(in_features=2048, out_features=2, bias=True)
)

In [7]:
params = list(model.parameters())
params

[Parameter containing:
 tensor([[-0.0309, -0.0317, -0.0266,  ..., -0.0044, -0.0040,  0.0102],
         [ 0.0188,  0.0244, -0.0259,  ..., -0.0082, -0.0350, -0.0341],
         [-0.0110,  0.0200,  0.0020,  ...,  0.0342, -0.0208,  0.0045],
         ...,
         [ 0.0235,  0.0336,  0.0200,  ..., -0.0017, -0.0198, -0.0113],
         [-0.0165,  0.0328,  0.0077,  ...,  0.0001,  0.0263, -0.0006],
         [ 0.0057, -0.0331,  0.0068,  ..., -0.0151, -0.0203,  0.0296]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0317,  0.0346,  0.0257,  ..., -0.0191, -0.0171,  0.0311],
        requires_grad=True),
 Parameter containing:
 tensor([[-0.0141, -0.0169, -0.0053,  ...,  0.0053,  0.0053,  0.0123],
         [-0.0178,  0.0086,  0.0198,  ...,  0.0001, -0.0044,  0.0183]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0149, -0.0032], requires_grad=True)]

In [8]:
params[0].data = f_weights_float
params[1].data = f_bias_float
params[2].data = r_weights_float
params[3].data = r_bias_float

### Loading MNIST dataset

In [9]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
def mnist_dataset(batch_size, train=True, values=list(range(10))):
    # Initializing MNIST data set.
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

    targets_list = dataset.targets.tolist()
    values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

    # Creating a subset of ### MNIST targets.
    subset = torch.utils.data.Subset(dataset, values_index)
    loader = DataLoader(dataset=subset, shuffle=True)

    return loader

In [10]:
MNIST = rs.mnist_dataset(batch_size=0, train=True, values=[0,1])
data, targets = next(iter(MNIST))

Next, in order to perform the CKA calc. we will need to reshape the data into a batch_size X features tenso (12665, 784).

In [11]:
data = torch.squeeze(data, dim=1)
data = data.view(data.size(0), -1)
data.shape

torch.Size([12665, 784])

### CKA Calc.
At this point we can now calculate the CKA for the model state.

In [12]:
model.features(data).shape

torch.Size([12665, 1024])

In [13]:
targets.shape

torch.Size([12665])

In [14]:
rs.kernel_calc(targets, model.features(data))

tensor(0.9521, grad_fn=<DivBackward0>)

#TODO using this notebook create a python script that will do this for each of the model states (135 * 512)...

### Accuracy

In [15]:
import torch.optim as optim

In [16]:
device = torch.device('cpu')
loss = nn.MSELoss()
model.eval()
losses = rs.train(MNIST, device, model, loss, values=[0, 1], backwards=False, record_loss=True)

In [17]:
losses

8.250704922829755e-06

In [18]:
import time

### CKA Test.
Defining Center Kernel Alignment functions.

In [83]:
def kernel_calc(y, phi):
    y = torch.t(torch.unsqueeze(y, -1))
    
    start = time.time()
    K1 = torch.matmul(torch.t(y), y)
    
    K1c = kernel_centering(K1.float())

    K2 = torch.mm(phi, torch.t(phi))
    
    K2c = kernel_centering(K2)
    end = time.time()

    return kernel_alignment(K1c, K2c)


def frobenius_product(K1, K2):
    return torch.sum(K1 * K2)


def kernel_alignment(K1, K2):
    inner = frobenius_product(K1, K2) 
    mag_norm = ((torch.norm(K1, p='fro') * torch.norm(K2, p='fro')))
    return inner / mag_norm


def kernel_centering(K):
    row_means = K.mean(dim=1, keepdim=True)
    col_means = K.mean(dim=0, keepdim=True)
    total_mean = K.mean()
    
    return K - row_means - col_means + total_mean

In [84]:
start = time.time()
cka = kernel_calc(targets, model.features(data))
end = time.time()
print(f"CKA : {cka} |TOTAL TIME: {end - start}s")

CKA : 0.9520593881607056 |TOTAL TIME: 7.976164102554321s


### New CKA Function

In [85]:
phi = model.features(data)
y = torch.unsqueeze(targets.T, -1)

The new methods potentially allows us to ignore the centering calculation.

In [105]:
# New Method
start = time.time()
v = phi.T.matmul(y.float())
inner = v.T @ v
end = time.time()
print("NEW METHOD")
print(f"INNER = {inner} | TIME = {end - start}s")

NEW METHOD
INNER = tensor([[4.0601e+08]], grad_fn=<MmBackward0>) | TIME = 0.0040819644927978516s


In [104]:
# Old Method
start = time.time()
K1 = y @ y.T
K1 = K1.float()
K2 = phi @ phi.T
inner = torch.trace(torch.mm(K2, torch.t(K1)))
end = time.time()
print("OLD METHOD")
print(f"INNER = {inner} | TIME = {end - start}s")

OLD METHOD
INNER = 406006880.0 | TIME = 13.496126890182495s


In [ ]:
13.496126890182495 